<a href="https://colab.research.google.com/github/ny-yo/kaggle-CommonLit-Readability-Prize/blob/main/bert_sturdy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .

fatal: destination path 'transformers' already exists and is not an empty directory.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2308934 sha256=7254d1beaa771cde181664f43a3ba601b006dd2c4bcd2d8615c254e50febd1a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-yxdua_8z/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 4.7.0.dev0
    Uninstalling transformers-4.7.0.dev0:
      Successfully uninstalled transformers-4.7.0.dev0


In [3]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install fugashi ipadic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.8)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.13)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.4)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already instal

In [4]:
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
import os
import urllib.request
import re
import csv
import tarfile
import transformers
import numpy as np
# データのダウンロード（カレントディレクトリに圧縮ファイルがダウンロードされる）
urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")

('ldcc-20140209.tar.gz', <http.client.HTTPMessage at 0x7f28c1dfa810>)

In [6]:
# ダウンロードした圧縮ファイルのパスを設定
tgz_fname = "ldcc-20140209.tar.gz" 
# 2つをニュースメディアのジャンルを選定
target_genre = ["it-life-hack", "sports-watch"] 
#処理をした結果を保存するファイル名 
tsv_fname = "all_text.tsv" 


# 処理部分-------
brackets_tail = re.compile('【[^】]*】$')
brackets_head = re.compile('^【[^】]*】')

def remove_brackets(inp):
    output = re.sub(brackets_head, '', re.sub(brackets_tail, '', inp))
    return output

def read_title(f):
    # 2行スキップ
    next(f)
    next(f)
    title = next(f) # 3行目を返す
    title = remove_brackets(title.decode('utf-8'))
    return title[:-1]

zero_fnames = []
one_fnames = []

with tarfile.open(tgz_fname) as tf:
    # 対象ファイルの選定
    for ti in tf:
        # ライセンスファイルはスキップ
        if "LICENSE.txt" in ti.name:
            continue
        if target_genre[0] in ti.name and ti.name.endswith(".txt"):
            zero_fnames.append(ti.name)
            continue
        if target_genre[1] in ti.name and ti.name.endswith(".txt"):
            one_fnames.append(ti.name)
    with open(tsv_fname, "w") as wf:
        writer = csv.writer(wf, delimiter='\t')
        # ラベル 0
        for name in zero_fnames:
            f = tf.extractfile(name)
            title = read_title(f)
            row = [target_genre[0], 0, '', title]
            writer.writerow(row)
        # ラベル 1
        for name in one_fnames:
            f = tf.extractfile(name)
            title = read_title(f)
            row = [target_genre[1], 1, '', title]
            writer.writerow(row)

In [7]:
import pandas as pd

#データの読み込み
df = pd.read_csv("all_text.tsv", delimiter='\t', header=None, names=['media_name', 'label', 'NaN', 'sentence'])

#データの確認
print(f'データサイズ： {df.shape}')
df.sample(10)

データサイズ： (1770, 4)


,media_name,label,NaN,sentence
1211,sports-watch,1,NaN,内田、背水のセカンドレグへ「こんなところで投げたくない」
879,sports-watch,1,NaN,決戦から一夜、真央「3回飛ぶって決めてました」
205,it-life-hack,0,NaN,おはこんです！紺子のＭＭＤ動画が流れる予定ですっ
1586,sports-watch,1,NaN,横浜・中畑監督を痛烈に批判、「精神面を問題にするようじゃ最下位」
1656,sports-watch,1,NaN,“テコンドー界のエリカ様”平成生まれの美少女アスリート＝笠原江梨香に注目集まる
1594,sports-watch,1,NaN,ダル実戦デビューに「あれが本物のダルビッシュだと思ったら大きな間違い」
299,it-life-hack,0,NaN,もちろんカレー臭はしなかった！ 品質もそこそこなインド製携帯電話
364,it-life-hack,0,NaN,Twitterの影響力マジパねぇ！ 東大駒場校舎で行われた無料耳かきサービスに人だかり
661,it-life-hack,0,NaN,孫社長が新スマホを大々的に宣伝！放射線測定スマホを開発した理由
27,it-life-hack,0,NaN,タダ者じゃない！7インチタブレット「eden TAB」の魅力をさぐる


In [8]:
# データの抽出
sentences = df.sentence.values
labels = df.label.values

In [9]:
# 1. BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


In [10]:
## テスト実行
# 元文章
print(' Original: ', sentences[0])
# Tokenizer
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
# Token-id
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック
Tokenized:  ['旧式', 'Mac', 'で', '禁', '##断', 'の', 'パワーアップ', '!', '最新', 'PC', 'や', 'ソフト', 'を', '一挙', 'に', 'チェック']
Token IDs:  [18718, 8653, 12, 1763, 29135, 5, 20734, 679, 6215, 3794, 49, 1604, 11, 24598, 7, 9398]


In [11]:
#最大単語数の確認
max_len = []

#1文づつ処理
for sent in sentences:
  #tokenizerで分割
  token_words = tokenizer.tokenize(sent)
  #文章数を取得してリストへ格納
  max_len.append(len(token_words))
#最大の数を確認
print('最大単語数：', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

最大単語数： 35
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [12]:
input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = 37,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック
Token IDs: tensor([    2, 18718,  8653,    12,  1763, 29135,     5, 20734,   679,  6215,
         3794,    49,  1604,    11, 24598,     7,  9398,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])


ここまでは前処理を実施していたので、90%を訓練データ、10%をテストデータに分割して、pytorchで学習を行うためにデータローダーへ変換する

In [13]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

#データセットクラスの作成 trainとtestをまとめる
#https://kento1109.hatenablog.com/entry/2018/02/19/144109
dataset = TensorDataset(input_ids, attention_masks, labels)

#90%地点のIDを取得
train_size = int(0.9*len(dataset))
val_size = len(dataset) - train_size

#データセットを分割する
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数：{}'.format(val_size))

#データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )


訓練データ数：1593
検証データ数：177


学習済みモデルロード

In [14]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 2, # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

訓練(Pre-trainedモデルのファインチューニング)

In [15]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs= model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            val_loss += loss.item()
    return val_loss

学習に必要な関数が定義できたので、学習を行う

In [16]:
# 学習の実行
max_epoch = 4
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = train(model)
    train_loss_.append(train_)
    test_loss_.append(test_)

検証

In [17]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

In [18]:
## 予測結果の確認
print(f'出力:{preds}')

出力:SequenceClassifierOutput(loss=None, logits=tensor([[ 3.8569, -5.1612],
        [ 4.1836, -5.6198],
        [-3.7384,  5.3962],
        [-4.3365,  5.6296],
        [-4.3556,  5.5643],
        [ 4.1479, -5.5610],
        [-4.3778,  5.5701],
        [-4.3649,  5.5774],
        [ 4.1382, -5.6746],
        [-4.3021,  5.5961],
        [ 3.9430, -5.5044],
        [ 4.0902, -5.6656],
        [ 4.1682, -5.5363],
        [ 4.1337, -5.5054],
        [-4.3386,  5.6267],
        [-4.2808,  5.5971],
        [ 4.1567, -5.6577]], device='cuda:0'), hidden_states=None, attentions=None)


In [19]:
# 比較しやすい様にpd.dataframeへ整形
import pandas as pd
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

,logit_0,logit_1,pred_label,true_label
0,3.856918,-5.161248,0,0
1,4.183552,-5.619831,0,0
2,-3.738379,5.396243,1,1
3,-4.336545,5.629647,1,1
4,-4.355567,5.564263,1,1
